In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from mlxtend.classifier import StackingCVClassifier
import copy

train = pd.read_csv("./cat-in-the-dat/train.csv", index_col=['id'])
test = pd.read_csv("./cat-in-the-dat/test.csv", index_col=['id'])

In [3]:
train.dtypes

bin_0      int64
bin_1      int64
bin_2      int64
bin_3     object
bin_4     object
nom_0     object
nom_1     object
nom_2     object
nom_3     object
nom_4     object
nom_5     object
nom_6     object
nom_7     object
nom_8     object
nom_9     object
ord_0      int64
ord_1     object
ord_2     object
ord_3     object
ord_4     object
ord_5     object
day        int64
month      int64
target     int64
dtype: object

In [4]:
display(train.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [39]:
X = train.drop("target", axis = 1)
y = train.loc[:,"target"]

### Encoding for catagory 

#### Binary data

In [184]:
X.bin_3 = X.bin_3.apply(lambda x: 1 if x == "T" else 0)
X.bin_4 = X.bin_4.apply(lambda x: 1 if x == "Y" else 0)

# X = X.drop(["nom_0", "nom_1", "nom_2", "nom_3", "nom_4"], axis=1) \
#         .join(pd.get_dummies(X[["nom_0", "nom_1", "nom_2", "nom_3", "nom_4"]]))

X.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2',
       'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0',
       'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month'],
      dtype='object')

#### Nominal data

In [6]:
# h = FeatureHasher(input_type='string', n_features=1000)
# X[['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']].values
# hash_X = h.fit_transform(X[['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']].values)
# hash_X = pd.DataFrame(hash_X.toarray())

# hash_X.columns

# X = X.drop(["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"], axis=1).join(hash_X)

# loo_encoder = LeaveOneOutEncoder(cols=["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"])
# loo_X = loo_encoder.fit_transform(X[["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]], y)
# X = X.drop(["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"], axis=1).join(loo_X)

# X.columns

#### Ordinal data

In [149]:
# oe = OrdinalEncoder(categories='auto')
# df = pd.DataFrame({'foo1': ['a','c','a','a','b','d']})
# oe.fit_transform(df.foo1.values.reshape(-1,1))

array([[0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [3.]])

In [185]:
X.ord_1.replace(to_replace = ['Novice', 'Contributor','Expert', 'Master', 'Grandmaster'],
                         value = [0, 1, 2, 3, 4], inplace = True)
X.ord_2.replace(to_replace = ['Freezing', 'Cold', 'Warm', 'Hot','Boiling Hot', 'Lava Hot'],
                         value = [0, 1, 2, 3, 4, 5], inplace = True)

for i in ["ord_3", "ord_4"]:
   le = LabelEncoder()
   X[[i]] = le.fit_transform(X[[i]])

oe = OrdinalEncoder(categories='auto')
X.ord_5 = oe.fit_transform(X.ord_5.values.reshape(-1,1))

#X.columns

# X = X.drop(["ord_3", "ord_4", "ord_5"], axis=1) \
#         .join(pd.get_dummies(X[["ord_3", "ord_4", "ord_5"]]))

# X.columns


/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Timeseries data

In [16]:
X[["day","month"]] = X[["day","month"]].astype("object")
X.dtypes
#pd.get_dummies(X[["day", "month"]])

bin_0        int64
bin_1        int64
bin_2        int64
bin_3        int64
bin_4        int64
             ...  
ord_5_yY     uint8
ord_5_yc     uint8
ord_5_zU     uint8
day         object
month       object
Length: 1268, dtype: object

In [17]:
X = X.drop(["day", "month"], axis=1) \
        .join(pd.get_dummies(X[["day", "month"]]))

X.columns

#def date_cyc_enc(df, col, max_vals):
#    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
#    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
#    return df

#X = date_cyc_enc(X, 'day', 7)
#X = date_cyc_enc(X, 'month', 12)
#X.drop(['day', 'month'], axis=1, inplace = True)

#X.columns

Index([      'bin_0',       'bin_1',       'bin_2',       'bin_3',
             'bin_4',       'ord_0',       'ord_1',       'ord_2',
        'nom_0_Blue', 'nom_0_Green',
       ...
           'month_3',     'month_4',     'month_5',     'month_6',
           'month_7',     'month_8',     'month_9',    'month_10',
          'month_11',    'month_12'],
      dtype='object', length=1285)

### Scaling data

* Tree based models does not depend on scaling
* Non-tree based models hugely depend on scaling

So if I use linear model, scaling is essential, otherwise not.

In [111]:
#min_max_scaler = MinMaxScaler()
#X_minmax = min_max_scaler.fit_transform(X)

# scaler = StandardScaler()
# X_new = scaler.fit_transform(X)

MemoryError: Unable to allocate array with shape (300000, 8524) and data type float64

### Model refinement

#### Try it on linear model

In [202]:
lr = LogisticRegression()
scores_lr = cross_val_score(lr, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_lr.mean(), scores_lr.std() * 2))

Accuracy: 0.76 (+/- 0.00)


In [174]:
rc = RidgeClassifier()
scores_rc = cross_val_score(rc, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rc.mean(), scores_rc.std() * 2))

Accuracy: 1.00 (+/- 0.00)


In [41]:
lda = LinearDiscriminantAnalysis()
scores_lda = cross_val_score(lda, X_new, y, cv=5, n_jobs=1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_lda.mean(), scores_lda.std() * 2))

/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy: 0.76 (+/- 0.00)


In [ ]:
linear_svm = LinearSVC(penalty="l2")
scores_linear_svm = cross_val_score(linear_svm, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_linear_svm.mean(), scores_linear_svm.std() * 2))

LogisticRegression, RidgeClassifier and LinearDiscriminantAnalysis revel better accuracy than LinearSVC. But if use X insteading of X_new, weather it has still good accuracy.

We can see that scaled data maybe has better accuracy original data.

#### Try it on classification model

In [22]:
fr = DecisionTreeClassifier(random_state=0)
scores_dt = cross_val_score(fr, X, y, cv=5, n_jobs=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt.mean(), scores_dt.std() * 2))

Accuracy: 0.64 (+/- 0.04)


In [23]:
sgdc = SGDClassifier()
scores_sgdc = cross_val_score(sgdc, X, y, cv=5, n_jobs=4)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_sgdc.mean(), scores_sgdc.std() * 2))

Accuracy: 0.46 (+/- 0.38)


In [74]:
ab = AdaBoostClassifier()
scores_ab= cross_val_score(ab, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_ab.mean(), scores_ab.std() * 2))

Accuracy: 0.76 (+/- 0.00)


In [75]:
gbm = GradientBoostingClassifier()
scores_gbm= cross_val_score(gbm, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_gbm.mean(), scores_gbm.std() * 2))

Accuracy: 0.77 (+/- 0.01)


In [77]:
rf = RandomForestClassifier()
scores_rf= cross_val_score(rf, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std() * 2))

Accuracy: 0.73 (+/- 0.00)


In [78]:
et = ExtraTreesClassifier()
scores_et= cross_val_score(et, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_et.mean(), scores_et.std() * 2))

Accuracy: 0.72 (+/- 0.00)


In [79]:
xgb = XGBClassifier()
scores_xgb= cross_val_score(xgb, X, y, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_xgb.mean(), scores_xgb.std() * 2))

Accuracy: 0.77 (+/- 0.00)


We can see that GradientBoostingClassifier, XGBClassifier maybe have the best accuracy in above methods, which the accuracy of AdaBoostClassifier is also close to. Otherwise, others maybe performent not good.

#### Grid Search for best params of XGBClassifier model

In [80]:
params = {
        'min_child_weight': [1, 5, 10, 13, 15],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.2, 0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 10, 20]
        }
xgb = XGBClassifier(silent=True, nthread=1)
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, 
                                   scoring='accuracy', n_jobs=-1, cv=skf.split(X, y), 
                                   verbose=3, random_state=1001 )
random_search.fit(X, y)

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
#results = pd.DataFrame(random_search.cv_results_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   55.4s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  2.1min remaining:   32.2s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.5min finished



 All results:
{'mean_fit_time': array([ 47.79684703,  52.27401908, 119.7774841 ,  48.17668215,
        57.93815573]), 'std_fit_time': array([ 0.29789206,  0.95753492, 14.84604595,  0.37390398, 12.90913243]), 'mean_score_time': array([0.47573551, 0.54289651, 0.91353989, 0.53941655, 0.51821582]), 'std_score_time': array([0.02845468, 0.00212782, 0.04051057, 0.00586698, 0.07428489]), 'param_subsample': masked_array(data=[0.6, 0.2, 0.8, 0.4, 0.2],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 5, 10, 10, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 4, 10, 4, 5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1.5, 1.5, 5, 1.5, 1.5],
             mask=[False, False, False, False, False],
      

In [81]:
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

0.768 (+/-0.003) for {'subsample': 0.6, 'min_child_weight': 1, 'max_depth': 3, 'gamma': 1.5, 'colsample_bytree': 0.8}
0.788 (+/-0.006) for {'subsample': 0.2, 'min_child_weight': 5, 'max_depth': 4, 'gamma': 1.5, 'colsample_bytree': 0.8}
0.868 (+/-0.010) for {'subsample': 0.8, 'min_child_weight': 10, 'max_depth': 10, 'gamma': 5, 'colsample_bytree': 0.8}
0.791 (+/-0.008) for {'subsample': 0.4, 'min_child_weight': 10, 'max_depth': 4, 'gamma': 1.5, 'colsample_bytree': 0.6}
0.799 (+/-0.002) for {'subsample': 0.2, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 1.5, 'colsample_bytree': 1.0}


#### Grid Search for best params of GradientBoostingClassifier model

In [31]:
params2 = {
        'n_estimators': [50, 100, 300, 800],
        'learning_rate': [0.01, 0.1, 0.5, 1],
        'max_depth': [3, 10, 20, 50],
        'min_samples_split': [100, 200, 500, 800],
        'subsample': [0.2, 0.4, 0.6, 0.8, 1.0]
        }
gbm = GradientBoostingClassifier(random_state=1001)

random_search2 = RandomizedSearchCV(gbm, param_distributions=params2, n_iter=param_comb, 
                                   scoring='accuracy', n_jobs=-1, cv=skf.split(X, y), 
                                   verbose=3, random_state=1001 )
random_search2.fit(X, y)

print('\n All results:')
print(random_search2.cv_results_)
print('\n Best estimator:')
print(random_search2.best_estimator_)
print('\n Best hyperparameters:')
print(random_search2.best_params_)
#results = pd.DataFrame(random_search2.cv_results_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed: 67.2min finished



 All results:
{'mean_fit_time': array([ 908.29501446,  845.39744933, 1648.9516054 ,  731.22689096,
        664.19118261]), 'std_fit_time': array([ 94.75944472,  85.08638074, 586.84364353,  32.08922876,
       115.58840373]), 'mean_score_time': array([1.55622586, 1.18005204, 1.39842073, 0.71798897, 0.64372404]), 'std_score_time': array([0.50675279, 0.00952958, 0.36244974, 0.00273974, 0.04241387]), 'param_subsample': masked_array(data=[0.6, 0.2, 0.8, 0.4, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[800, 100, 300, 100, 100],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_split': masked_array(data=[800, 200, 100, 500, 800],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[20, 50, 50, 50, 20],
             mask=[Fal

In [84]:
means = random_search2.cv_results_['mean_test_score']
stds = random_search2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

1.000 (+/-0.000) for {'subsample': 0.6, 'n_estimators': 800, 'min_samples_split': 800, 'max_depth': 20, 'learning_rate': 0.5}
0.932 (+/-0.032) for {'subsample': 0.2, 'n_estimators': 100, 'min_samples_split': 200, 'max_depth': 50, 'learning_rate': 0.5}
0.982 (+/-0.037) for {'subsample': 0.8, 'n_estimators': 300, 'min_samples_split': 100, 'max_depth': 50, 'learning_rate': 1}
0.725 (+/-0.001) for {'subsample': 0.4, 'n_estimators': 100, 'min_samples_split': 500, 'max_depth': 50, 'learning_rate': 0.01}
0.723 (+/-0.001) for {'subsample': 0.6, 'n_estimators': 100, 'min_samples_split': 800, 'max_depth': 20, 'learning_rate': 0.01}


#### Grid Search for best params of LogisticRegression model

In [85]:
lr_params = {'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
             'C': [0.01, 0.1, 0.5, 1]
            }
lr = LogisticRegression(random_state=1001)

random_search3 = RandomizedSearchCV(lr, param_distributions=lr_params, n_iter=param_comb, 
                                   scoring='accuracy', n_jobs=-1, cv=skf.split(X, y), 
                                   verbose=3, random_state=1001 )
random_search3.fit(X, y)

print('\n All results:')
print(random_search3.cv_results_)
print('\n Best estimator:')
print(random_search3.best_estimator_)
print('\n Best hyperparameters:')
print(random_search3.best_params_)
#results = pd.DataFrame(random_search2.cv_results_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   28.7s remaining:   43.0s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   40.5s remaining:   10.1s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   44.6s finished



 All results:
{'mean_fit_time': array([ 8.74004189, 29.41071987, 24.10780462, 22.03874866, 16.1790061 ]), 'std_fit_time': array([0.8806604 , 0.95997239, 1.08362148, 4.4700542 , 0.78093431]), 'mean_score_time': array([0.04158473, 0.02806997, 0.03090366, 0.0189476 , 0.01525704]), 'std_score_time': array([0.00647106, 0.00441646, 0.00413027, 0.00091417, 0.00100142]), 'param_solver': masked_array(data=['lbfgs', 'sag', 'newton-cg', 'sag', 'saga'],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_C': masked_array(data=[0.01, 0.5, 0.01, 0.01, 0.1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'solver': 'lbfgs', 'C': 0.01}, {'solver': 'sag', 'C': 0.5}, {'solver': 'newton-cg', 'C': 0.01}, {'solver': 'sag', 'C': 0.01}, {'solver': 'saga', 'C': 0.1}], 'split0_test_score': array([0.74299, 0.75941, 0.75857, 0.75665, 0.75495]), 'split1_test_score': array([0.73967, 0.7

/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [86]:
means = random_search3.cv_results_['mean_test_score']
stds = random_search3.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search3.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

0.741 (+/-0.003) for {'solver': 'lbfgs', 'C': 0.01}
0.760 (+/-0.000) for {'solver': 'sag', 'C': 0.5}
0.759 (+/-0.001) for {'solver': 'newton-cg', 'C': 0.01}
0.757 (+/-0.001) for {'solver': 'sag', 'C': 0.01}
0.755 (+/-0.002) for {'solver': 'saga', 'C': 0.1}


### Stacked model

In [87]:
xgb_clf = XGBClassifier(booster='gbtree', gamma=5, colsample_bytree=0.8,
                        learning_rate=0.1, max_depth=10, 
                        min_child_weight=10, n_estimators=100, 
                        silent=True, subsample=0.8)

ab_clf = AdaBoostClassifier(n_estimators=200,
                            base_estimator=DecisionTreeClassifier(
                                min_samples_leaf=2,
                                random_state=1001),
                            random_state=1001)

gbm_clf = GradientBoostingClassifier(n_estimators=300, min_samples_split=100,
                                 max_depth=50, learning_rate=1, subsample=0.8,
                                 random_state=1001)

lr = LogisticRegression()

stack = StackingCVClassifier(classifiers=[xgb_clf, gbm_clf, ab_clf], 
                            meta_classifier=lr,
                            cv=5,
                            stratify=True,
                            shuffle=True,
                            use_probas=True,
                            use_features_in_secondary=True,
                            verbose=1,
                            random_state=1001,
                            n_jobs=-1)
stack = stack.fit(X, y)
#scores_stack = cross_val_score(stack, X, y, cv=2, n_jobs=-1, scoring='accuracy')
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores_stack.mean(), scores_stack.std() * 2))

Fitting 3 classifiers...
Fitting classifier1: xgbclassifier (1/3)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting classifier2: gradientboostingclassifier (2/3)


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 47.1min remaining: 70.6min


Fitting classifier3: adaboostclassifier (3/3)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 238.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/home/anlan/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 16.1min remaining: 24.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 20.1min finished
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Encoding for test data

In [38]:
X_test = copy.deepcopy(test)

display(X_test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [188]:
X_test.bin_3 = X_test.bin_3.apply(lambda x: 1 if x == "T" else 0)
X_test.bin_4 = X_test.bin_4.apply(lambda x: 1 if x == "Y" else 0)

# X_test = X_test.drop(["nom_0", "nom_1", "nom_2", "nom_3", "nom_4"], axis=1) \
#         .join(pd.get_dummies(X_test[["nom_0", "nom_1", "nom_2", "nom_3", "nom_4"]]))

# loo_X_test = loo_encoder.transform(X_test[["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]])
# X_test = X_test.drop(["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"], axis=1).join(loo_X_test)

X_test.ord_1.replace(to_replace = ['Novice', 'Contributor','Expert', 'Master', 'Grandmaster'],
                         value = [0, 1, 2, 3, 4], inplace = True)
X_test.ord_2.replace(to_replace = ['Freezing', 'Cold', 'Warm', 'Hot','Boiling Hot', 'Lava Hot'],
                         value = [0, 1, 2, 3, 4, 5], inplace = True)

# X_test = X_test.drop(["ord_3", "ord_4", "ord_5"], axis=1) \
#         .join(pd.get_dummies(X_test[["ord_3", "ord_4", "ord_5"]]))

# X_test = X_test.drop(["day", "month"], axis=1) \
#         .join(pd.get_dummies(X_test[["day", "month"]]))


for i in ["ord_3", "ord_4"]:
    le = LabelEncoder()
    X_test[[i]] = le.fit_transform(X_test[[i]])

oe = OrdinalEncoder(categories='auto')
X_test.ord_5 = oe.fit_transform(X_test.ord_5.values.reshape(-1,1))

# X_test = date_cyc_enc(X_test, 'day', 7)
# X_test = date_cyc_enc(X_test, 'month', 12)
# X_test.drop(['day', 'month'], axis=1, inplace = True)

/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
len(X_test.ord_5.value_counts())

192

In [72]:
len(X.ord_5.value_counts())

192

In [189]:
display(X_test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,1,1,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,0,2,9,15,95.0,5,11
300001,0,0,0,1,0,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,3,5,11,0,61.0,7,5
300002,1,0,1,0,1,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,2,0,0,6,172.0,1,12
300003,0,0,1,1,1,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,1,5,1,16,135.0,2,3
300004,0,1,1,0,0,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,4,5,11,22,157.0,4,11


In [40]:
print(X.shape)
print(X_test.shape)

(300000, 23)
(200000, 23)


In [41]:
data = pd.concat([X, X_test])

In [161]:
# data.drop("bin_0", axis=1, inplace=True)

In [42]:
print(data.shape)

(500000, 23)


In [99]:
# data = data.astype("object")

In [43]:
data.dtypes

bin_0     int64
bin_1     int64
bin_2     int64
bin_3    object
bin_4    object
nom_0    object
nom_1    object
nom_2    object
nom_3    object
nom_4    object
nom_5    object
nom_6    object
nom_7    object
nom_8    object
nom_9    object
ord_0     int64
ord_1    object
ord_2    object
ord_3    object
ord_4    object
ord_5    object
day       int64
month     int64
dtype: object

In [47]:
columns = data.columns
dummies = pd.get_dummies(data,
                         columns=columns,
#                          drop_first=True,
                         sparse=True)

In [48]:
print(dummies.shape)

(500000, 16552)


In [49]:
print(X.shape[0])

300000


In [50]:
X = dummies.iloc[:X.shape[0], :]
X_test = dummies.iloc[X.shape[0]:, :]

In [51]:
del dummies
del data
print(X.shape)
print(X_test.shape)

(300000, 16552)
(200000, 16552)


In [52]:
display(X.head())

,bin_0_0,bin_0_1,bin_1_0,bin_1_1,bin_2_0,bin_2_1,bin_3_F,bin_3_T,bin_4_N,bin_4_Y,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
id,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1,0,1,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [53]:
X = X.sparse.to_coo().tocsr()
X_test = X_test.sparse.to_coo().tocsr()

X = X.astype("float32")
X_test = X_test.astype("float32")

#### Making predictions

In [195]:
X_test2 = np.array(X_test)
#pred = stack.predict(X_test)
pred = stack.predict_proba(X_test2)

In [197]:
pred[:10,1]

array([1.60453924e-01, 4.15311686e-01, 9.71599955e-01, 1.90172725e-01,
       1.93344159e-04, 1.47929063e-03, 8.76056952e-03, 1.25936934e-02,
       9.90136422e-01, 4.93963888e-05])

In [199]:
display(X_test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,ord_0,ord_1,ord_2,ord_3,ord_4,...,nom_4_Theremin,nom_5,nom_6,nom_7,nom_8,nom_9,day_sin,day_cos,month_sin,month_cos
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,1,1,2,0,2,9,15,...,0,0.327116,0.330645,0.299065,0.348315,0.181818,-9.749279e-01,-0.222521,-5.000000e-01,8.660254e-01
300001,0,0,0,1,0,1,3,5,11,0,...,0,0.329803,0.399600,0.389796,0.222707,0.288889,-2.449294e-16,1.000000,5.000000e-01,-8.660254e-01
300002,1,0,1,0,1,2,2,0,0,6,...,0,0.336677,0.305964,0.317597,0.186667,0.090909,7.818315e-01,0.623490,-2.449294e-16,1.000000e+00
300003,0,0,1,1,1,1,1,5,1,16,...,0,0.328419,0.244373,0.361022,0.360656,0.320000,9.749279e-01,-0.222521,1.000000e+00,6.123234e-17
300004,0,1,1,0,0,3,4,5,11,22,...,0,0.243365,0.330569,0.271300,0.375000,0.294118,-4.338837e-01,-0.900969,-5.000000e-01,8.660254e-01


In [174]:
xgb_clf2 = XGBClassifier(booster='gbtree', gamma=5, colsample_bytree=0.8,
                        learning_rate=0.1, max_depth=10, 
                        min_child_weight=10, n_estimators=100, 
                        silent=True, subsample=0.8, n_jobs=-1)
xgb_clf2.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=5,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)

In [176]:
#X_test = np.array(X_test)

In [181]:
pred2 = xgb_clf2.predict(X_test)
pred2[:10]

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

In [184]:
gbm_clf2 = GradientBoostingClassifier(n_estimators=300, min_samples_split=100,
                                 max_depth=50, learning_rate=1, subsample=0.8,
                                 random_state=1001)
gbm_clf2.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None, learning_rate=1,
                           loss='deviance', max_depth=50, max_features=None,
                           max_leaf_nodes=None, min_impurity_decrease=0.0,
                           min_impurity_split=None, min_samples_leaf=1,
                           min_samples_split=100, min_weight_fraction_leaf=0.0,
                           n_estimators=300, n_iter_no_change=None,
                           presort='auto', random_state=1001, subsample=0.8,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [185]:
pred3 = gbm_clf2.predict(X_test)
pred3[:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [189]:
pipe.fit(X, y)

/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='warn', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [201]:
pred4 = pipe.predict_proba(X_test)
pred4[:10,1]

array([0.23799456, 0.65998172, 0.07750979, 0.47985296, 0.82529916,
       0.16685068, 0.4914223 , 0.16917902, 0.44640971, 0.51153191])

In [209]:
#X.drop(["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"])
#len(X.columns)
X.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'ord_0', 'ord_1', 'ord_2',
       'ord_3', 'ord_4', 'ord_5', 'nom_0_Blue', 'nom_0_Green', 'nom_0_Red',
       'nom_1_Circle', 'nom_1_Polygon', 'nom_1_Square', 'nom_1_Star',
       'nom_1_Trapezoid', 'nom_1_Triangle', 'nom_2_Axolotl', 'nom_2_Cat',
       'nom_2_Dog', 'nom_2_Hamster', 'nom_2_Lion', 'nom_2_Snake',
       'nom_3_Canada', 'nom_3_China', 'nom_3_Costa Rica', 'nom_3_Finland',
       'nom_3_India', 'nom_3_Russia', 'nom_4_Bassoon', 'nom_4_Oboe',
       'nom_4_Piano', 'nom_4_Theremin', 'nom_5', 'nom_6', 'nom_7', 'nom_8',
       'nom_9', 'day_sin', 'day_cos', 'month_sin', 'month_cos'],
      dtype='object')

In [3]:
#X.columns[:36]
#display(X[["bin_0","bin_1"]].head())
#colnames = X.columns[:36]
display(X[X.columns[:36]].head())


NameError: name 'X' is not defined

In [235]:
from sklearn.linear_model import LogisticRegressionCV
#lr = LogisticRegression(penalty="l1")
lr = LogisticRegressionCV()
lr.fit(X, y)
pred5 = lr.predict_proba(X_test)
#lr.fit(X[X.columns[:36]], y)
#pred5 = lr.predict_proba(X_test[X_test.columns[:36]])
pred5[:10,1]

/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs

array([0.23709502, 0.6487434 , 0.07883924, 0.47918679, 0.82513343,
       0.17479619, 0.49127036, 0.16852144, 0.44993832, 0.50750788])

In [18]:
#from sklearn.linear_model import LogisticRegressionCV

pipe1 = make_pipeline(StandardScaler(), LogisticRegression(solver="lbfgs", C=0.1, max_iter=10000))

pipe2 = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis())

#pipe3 = make_pipeline(StandardScaler(), RidgeClassifier())
pipe3 = make_pipeline(StandardScaler(), LogisticRegressionCV())

stack = StackingCVClassifier(classifiers=[pipe2, pipe3], 
                            meta_classifier=pipe1,
                            cv=5,
                            stratify=True,
                            shuffle=True,
                            use_probas=True,
                            use_features_in_secondary=True,
                            verbose=1,
                            random_state=1001,
                            n_jobs=-1)
stack = stack.fit(X, y)

Fitting 2 classifiers...
Fitting classifier1: pipeline (1/2)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    5.7s remaining:    8.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.9s finished


Fitting classifier2: pipeline (2/2)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    8.8s remaining:   13.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   10.0s finished
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/anlan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [19]:
pred6 = stack.predict_proba(X_test)
pred6[:10,1]

array([0.24753769, 0.64842445, 0.07136907, 0.48044456, 0.81981137,
       0.17072148, 0.49109343, 0.1729329 , 0.45016073, 0.50994484])

In [55]:
pipe1 = make_pipeline(LogisticRegression(solver="lbfgs", C=0.1, max_iter=10000))
pipe1.fit(X, y)
pred7 = pipe1.predict_proba(X_test)
pred7[:10,1]

array([0.31251142, 0.5769731 , 0.09751631, 0.46268508, 0.8777967 ,
       0.14889053, 0.51918307, 0.1657425 , 0.39592094, 0.50175138])

In [54]:
X.columns
X_test.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'ord_0', 'ord_1', 'ord_2',
       'nom_0_Blue', 'nom_0_Green',
       ...
       'ord_5_wu', 'ord_5_wy', 'ord_5_xP', 'ord_5_xy', 'ord_5_yN', 'ord_5_yY',
       'ord_5_yc', 'ord_5_zU', 'day', 'month'],
      dtype='object', length=273)

In [61]:
X.bin_0.value_counts()

0    261749
1     38251
Name: bin_0, dtype: int64

In [62]:
y.value_counts()

0    208236
1     91764
Name: target, dtype: int64

In [45]:
columns = X_test.columns
X_test = pd.get_dummies(X_test,
                   columns=columns,
                   drop_first=True,
                   sparse=True)
X_test = X_test.sparse.to_coo().tocsr()
X_test = X_test.astype("float32")

In [52]:
print(X.shape)
print(X_test.shape)

(300000, 16438)
(200000, 16294)


In [209]:
pipe8 = make_pipeline(LogisticRegression())
pipe8.fit(X, y)
pred8 = pipe8.predict_proba(X_test)
pred8[:10,1]

array([0.29431422, 0.69282506, 0.07316665, 0.38696786, 0.90706751,
       0.17576176, 0.61499651, 0.16726831, 0.35722841, 0.39207942])

In [36]:
lr = LogisticRegression(solver="lbfgs", C=0.1, max_iter=10000)
lr.fit(X, y)
pred9 = lr.predict_proba(X_test)
pred9[:10,1]

array([0.33811315, 0.69877656, 0.12599665, 0.42130556, 0.86118582,
       0.16536373, 0.47803139, 0.17105752, 0.45641044, 0.40366503])

In [54]:
lr = LogisticRegression(solver="lbfgs", C=0.1, max_iter=10000)
lr.fit(X, y)
pred10 = lr.predict_proba(X_test)
pred10[:10,1]

array([0.34115138, 0.68599922, 0.12436068, 0.42224041, 0.86394186,
       0.16683914, 0.48127332, 0.15989305, 0.45831126, 0.4061595 ])

In [48]:
labels = train.pop('target')
labels = labels.values
# train_id = train.pop("id")
# test_id = test.pop("id")

data = pd.concat([train, test])
data["ord_5a"] = data["ord_5"].str[0]
data["ord_5b"] = data["ord_5"].str[1]
data.drop(["bin_0", "ord_5"], axis=1, inplace=True)

In [49]:
columns = [i for i in data.columns]

dummies = pd.get_dummies(data,
                         columns=columns,
                         drop_first=True,
                         sparse=True)

del data

In [50]:
train = dummies.iloc[:train.shape[0], :]
test = dummies.iloc[train.shape[0]:, :]

del dummies

In [51]:
print(train.shape)
print(test.shape)

(300000, 16438)
(200000, 16438)


#### Export predictions and submission

In [55]:
#X_test = pd.read_csv("./cat-in-the-dat/test.csv", index_col='id')
predictions = pd.Series(pred10[:,1], index=test.index, dtype=y.dtype)
predictions.to_csv("./cat-in-the-dat/submission10.csv", header=['target'], index=True, index_label='id')